<a href="https://colab.research.google.com/github/arturbernardo/chess_analysis/blob/main/chessanalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update
!apt-get install -y zstd

In [ ]:
file_path = '/content/lichess_db_standard_rated_2013-01.pgn.zst'
output_file_path = '/content/lichess_db_standard_rated_2013-01.pgn'
!zstd -d {file_path} -o {output_file_path}
print(f"File decompressed to: {output_file_path}")

/content/lichess_db_standard_rated_2013-01.pgn.zst: 92811021 bytes 
File decompressed to: /content/lichess_db_standard_rated_2013-01.pgn


In [ ]:
!pip install python-chess
print("python-chess library installed.")

python-chess library installed.


In [ ]:
# get_ipython().system('pip install python-chess')
# print("python-chess library installed.")

In [ ]:
import chess.pgn

pgn_file_path = '/content/lichess_db_standard_rated_2013-01.pgn'

with open(pgn_file_path, encoding="utf-8") as pgn_file:
    first_game = chess.pgn.read_game(pgn_file)
    print(first_game.headers)
    print(first_game.headers["White"])
    print(first_game.headers["Black"])
    print(first_game.headers["Result"])

    node = first_game
    while node.variations:
        next_node = node.variation(0)
        move = next_node.move
        print(node.board().san(move))  # Move em SAN (ex: Nf3, e4)
        node = next_node


Headers(Event='Rated Classical game', Site='https://lichess.org/j1dkb5dw', Date='????.??.??', Round='?', White='BFG9k', Black='mamalak', Result='1-0', UTCDate='2012.12.31', UTCTime='23:01:03', WhiteElo='1639', BlackElo='1403', WhiteRatingDiff='+5', BlackRatingDiff='-8', ECO='C00', Opening='French Defense: Normal Variation', TimeControl='600+8', Termination='Normal')
BFG9k
mamalak
1-0
e4
e6
d4
b6
a3
Bb7
Nc3
Nh6
Bxh6
gxh6
Be2
Qg5
Bg4
h5
Nf3
Qg6
Nh4
Qg5
Bxh5
Qxh4
Qf3
Kd8
Qxf7
Nc6
Qe8#


In [ ]:
selected_game = first_game

# INSTALL STOCKFISH

In [ ]:
# !apt-get update
# !apt-get install -y stockfish
# print("Stockfish installed.")

In [ ]:
#Downloaded
!wget https://www.dropbox.com/sh/75gzfgu7qo94pvh/AACk_w5M94GTwwhSItCqsemoa/Stockfish%205/stockfish-5-linux.zip
!unzip stockfish-5-linux.zip
!chmod +x stockfish-5-linux/Linux/stockfish_14053109_x64

stockfish_path = 'stockfish-5-linux/Linux/stockfish_14053109_x64'


In [ ]:
# engineb.quit

import chess.engine

engine = None
try:
    engine = chess.engine.SimpleEngine.popen_uci(stockfish_path)
    print(f"Stockfish engine successfully initialized. Name: {engine.id}")
except Exception as e:
    print(f"Error initializing Stockfish engine: {e}")

Stockfish engine successfully initialized. Name: {'name': 'Stockfish 5 64', 'author': 'Tord Romstad, Marco Costalba and Joona Kiiski'}


In [ ]:
# import chess.engine
# import asyncio

# async def initialize_stockfish():
#     global engine
#     engine = None
#     try:
#         transport, engine = await chess.engine.popen_uci(stockfish_path)
#         # The engine's name is typically in the 'id' dictionary
#         print(f"Stockfish engine successfully initialized. Name: {engine.id['name']}")
#     except Exception as e:
#         print(f"Error initializing Stockfish engine: {e}")

# # Run the async function
# await initialize_stockfish()

Stockfish engine successfully initialized. Name: Stockfish 5 64


## Analyze Single Game with Stockfish


In [ ]:
import chess
import chess.engine
import asyncio
import nest_asyncio

# Apply nest_asyncio to allow running asyncio event loops in Jupyter environment
nest_asyncio.apply()

# Helper to convert Score object to centipawns, handling mate scores
def get_score_in_centipawns(score_obj):
    if score_obj.is_mate():
        # Assign a high centipawn value for mate to represent its impact
        return 10000 if score_obj.mate() > 0 else -10000
    print(score_obj)
    return score_obj.score(mate_score=10000)

async def analyze_single_game(game, engine_instance):
    board = game.board()
    player_moves = []
    stockfish_moves = []
    stockfish_evaluations = [] # From White's perspective
    deviation_scores = [] # Centipawn Loss (CPL) for the player who just moved

    # Ensure the engine is running
    if engine_instance is None:
        print("Error: Stockfish engine is not initialized.")
        return [], [], [], []

    print(f"Starting analysis of game: {game.headers['Event']}")

    for i, move in enumerate(game.mainline_moves()):
        if board.is_game_over():
            break

        # 1. Analyze the current position with Stockfish to get the best move and its evaluation
        try:

            # result = engine.analyse(board, chess.engine.Limit(depth=10))
            # return result['score'].white().score()


            analysis_info_best = engine_instance.analyse(board, chess.engine.Limit(time=0.1)) # , multipv=1
            print(analysis_info_best['pv'])
            best_stockfish_move = analysis_info_best['pv'][0]
            print(best_stockfish_move)
            best_stockfish_score_obj = analysis_info_best['score']
            print(best_stockfish_score_obj)
            # Store evaluation from White's perspective
            stockfish_eval_white_pov = get_score_in_centipawns(best_stockfish_score_obj.pov(board.turn))
        except Exception as e:
            print(f"Error analyzing position (best move) at move {i+1}: {e}")
            break # Stop if engine fails

        # 2. Analyze the position *after* the player's actual move
        temp_board_after_player_move = board.copy()
        try:
            temp_board_after_player_move.push(move)
            analysis_info_player_move = engine_instance.analyse(temp_board_after_player_move, chess.engine.Limit(time=0.1))
            score_after_player_move_obj = analysis_info_player_move['score']
        except Exception as e:
            print(f"Error analyzing position (player move) at move {i+1}: {e}")
            # If the player move caused an error or was illegal, assume a large deviation
            # and proceed. This shouldn't happen with mainline_moves.
            score_after_player_move_obj = chess.engine.Cp(0) # Default to 0 CP if error

        # Calculate Centipawn Loss (CPL)
        # CPL = (Best score from current player's POV) - (Score after actual move from current player's POV)
        best_score_cp_current_player_pov = get_score_in_centipawns(best_stockfish_score_obj.pov(board.turn))
        score_after_player_move_cp_current_player_pov = get_score_in_centipawns(score_after_player_move_obj.pov(board.turn))
        cpl = best_score_cp_current_player_pov - score_after_player_move_cp_current_player_pov

        # Record the data
        player_moves.append(move.uci())
        stockfish_moves.append(best_stockfish_move.uci())
        stockfish_evaluations.append(stockfish_eval_white_pov)
        deviation_scores.append(cpl)

        # Apply the actual move to the main board to advance the game state
        board.push(move)

    # Close the engine after analysis is complete (if it's not needed for other tasks)
    # However, the task description says to close it *after the loop*, implying once per subtask
    # For this current subtask, it's just one game, so closing here is fine.
    # If the global 'engine' variable needs to be kept open for future steps,
    # we should move 'await engine.quit()' to the end of the main task.
    # For now, let's follow the instruction and close it.
    # await engine_instance.quit() # Moved to after calling analyze_single_game if needed
    print("Finished analyzing moves.")
    return player_moves, stockfish_moves, stockfish_evaluations, deviation_scores



engine = chess.engine.SimpleEngine.popen_uci(stockfish_path)
# Execute the analysis for the selected_game
# Need to ensure 'selected_game' and 'engine' (globally defined) are available
# The engine will be quit *after* this call using the global 'engine' object
player_moves, stockfish_moves, stockfish_evaluations, deviation_scores = await analyze_single_game(selected_game, engine)

# Quit the engine after completing all analysis for this task
engine.quit
print("Stockfish engine closed.")

print(f"Total moves analyzed: {len(player_moves)}")
print("Sample Player Moves:", player_moves[:5])
print("Sample Stockfish Moves:", stockfish_moves[:5])
print("Sample Stockfish Evaluations (CP, White POV):", stockfish_evaluations[:5])
print("Sample Centipawn Loss (CPL) per move:", deviation_scores[:5])


Starting analysis of game: Rated Classical game
[Move.from_uci('d2d4'), Move.from_uci('g8f6'), Move.from_uci('g1f3'), Move.from_uci('d7d5'), Move.from_uci('b1c3'), Move.from_uci('b8c6'), Move.from_uci('e2e3')]
d2d4
PovScore(Cp(+24), WHITE)
+24
+24
+40
[Move.from_uci('e7e6'), Move.from_uci('g1f3'), Move.from_uci('d7d5'), Move.from_uci('e4d5'), Move.from_uci('e6d5'), Move.from_uci('b1c3'), Move.from_uci('g8f6'), Move.from_uci('d2d4'), Move.from_uci('f8e7'), Move.from_uci('f1d3'), Move.from_uci('b8c6'), Move.from_uci('e1g1'), Move.from_uci('e8g8'), Move.from_uci('c1g5')]
e7e6
PovScore(Cp(-35), BLACK)
-35
-35
-48
[Move.from_uci('d2d4'), Move.from_uci('d7d5'), Move.from_uci('b1c3'), Move.from_uci('g8f6'), Move.from_uci('e4e5'), Move.from_uci('f6e4'), Move.from_uci('c3e4'), Move.from_uci('d5e4'), Move.from_uci('c2c3'), Move.from_uci('c7c5'), Move.from_uci('g1h3'), Move.from_uci('b8c6'), Move.from_uci('c1g5')]
d2d4
PovScore(Cp(+44), WHITE)
+44
+44
+41
[Move.from_uci('d7d5'), Move.from_uci('b1